In [1]:
!pip install transformers scikit-learn --quiet

In [17]:
!pip install sentence-transformers --quiet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 1.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 127.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 26.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 51.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 661.4 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 14.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 9.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 108.6 MB/s eta 0:00:00


In [18]:
from sentence_transformers import SentenceTransformer, util
from transformers import pipeline
import pandas as pd

In [19]:
documentos = {
    "turing": """Alan Turing fue un matemático británico que descifró el código Enigma en la Segunda Guerra Mundial.
                 También desarrolló la prueba de Turing para evaluar inteligencia artificial.""",
    "einstein": """Albert Einstein fue un físico alemán conocido por la teoría de la relatividad.
                   Su famosa fórmula es E=mc² y ganó el Nobel en 1921 por el efecto fotoeléctrico.""",
    "newton": """Isaac Newton formuló las leyes del movimiento y la gravedad universal.
                 Escribió el 'Principia Mathematica', una de las obras científicas más influyentes."""
}

In [20]:
pregunta = "¿Quién propuso una prueba para detectar si una máquina es inteligente?"

In [21]:
# Modelo de embeddings
model = SentenceTransformer('all-MiniLM-L6-v2')

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.5k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [22]:
# Embeddings de pregunta y documentos
docs = list(documentos.values())
doc_embs = model.encode(docs, convert_to_tensor=True)
query_emb = model.encode(pregunta, convert_to_tensor=True)

In [23]:
# Calcular similitud semántica
scores = util.cos_sim(query_emb, doc_embs)[0]
indice_max = scores.argmax().item()
nombre_doc = list(documentos.keys())[indice_max]
contexto = documentos[nombre_doc]

In [24]:
# QA
qa = pipeline("question-answering", model="distilbert-base-cased-distilled-squad")
respuesta = qa(question=pregunta, context=contexto)

Device set to use cpu


In [26]:
# Resultado final
df_resultado = pd.DataFrame([{
    "Documento seleccionado": nombre_doc,
    "Similitud semántica": round(scores[indice_max].item(), 4),
    "Respuesta": respuesta["answer"],
    "Confianza": round(respuesta["score"], 4)
}])

In [27]:
print(df_resultado)

  Documento seleccionado  Similitud semántica   Respuesta  Confianza
0                 turing               0.3992  artificial     0.1734


In [28]:
# Modelo de embeddings
model_embed = SentenceTransformer('all-MiniLM-L6-v2')
docs = list(documentos.values())
doc_embs = model_embed.encode(docs, convert_to_tensor=True)
query_emb = model_embed.encode(pregunta, convert_to_tensor=True)
scores = util.cos_sim(query_emb, doc_embs)[0]
indice_max = scores.argmax().item()
nombre_doc = list(documentos.keys())[indice_max]
contexto = documentos[nombre_doc]

In [29]:
# Modelos de QA a comparar
modelos_qa = {
    "distilbert": "distilbert-base-cased-distilled-squad",
    "bert": "bert-large-uncased-whole-word-masking-finetuned-squad",
    "roberta": "deepset/roberta-base-squad2",
    "electra": "deepset/electra-base-squad2"
}

In [30]:
# Evaluar cada modelo sobre el contexto seleccionado
resultados = []
for nombre, modelo in modelos_qa.items():
    qa = pipeline("question-answering", model=modelo, tokenizer=modelo)
    r = qa(question=pregunta, context=contexto)
    resultados.append({
        "Modelo QA": nombre,
        "Documento seleccionado": nombre_doc,
        "Similitud semántica": round(scores[indice_max].item(), 4),
        "Respuesta": r["answer"],
        "Confianza": round(r["score"], 4)
    })

Device set to use cpu


config.json:   0%|          | 0.00/443 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.34G [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-large-uncased-whole-word-masking-finetuned-squad were not used when initializing BertForQuestionAnswering: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForQuestionAnswering from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForQuestionAnswering from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

...

model.safetensors:   0%|          | 0.00/436M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/200 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

Device set to use cpu


In [31]:
# Mostrar resultados en DataFrame
df_resultados = pd.DataFrame(resultados)

In [33]:
df_resultados

,Modelo QA,Documento seleccionado,Similitud semántica,Respuesta,Confianza
0,distilbert,turing,0.3992,artificial,0.1734
1,bert,turing,0.3992,Alan Turing,0.3650
2,roberta,turing,0.3992,Alan Turing,0.0520
3,electra,turing,0.3992,Turing para evaluar inteligencia artificial,0.0297
